<a href="https://colab.research.google.com/github/YoungsikMoon/Find_Imo/blob/main/%EB%AC%B8%EC%98%81%EC%8B%9D/Imo_%EB%AC%B8%EC%98%81%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ViT 예시 코드

## 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
from einops import rearrange, repeat
from einops.layers.torch import Rearrange


## ViT 모델 구현

In [ ]:

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0.):
        super().__init__()
        image_height, image_width = image_size
        patch_height, patch_width = patch_size

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim=1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)



## 트랜스 포머 정의

In [ ]:
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x):
        return self.fn(self.norm(x))

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=h), qkv)

        dots = torch.einsum('b h i d, b h j d -> b h i j', q, k) * self.scale
        attn = dots.softmax(dim=-1)

        out = torch.einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


# ViT로 분류 구현해보기

## 필요 패키지 인스톨

In [ ]:
!pip install timm # 꼭 설치 하세요. 그리고 세션 재시작

## 필요 라이브러리 임포트

In [ ]:
# 필요한 라이브러리들을 임포트합니다.
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchvision.models import resnet50
from timm.models import vit_base_patch16_224

## 분류 코드 구현

In [ ]:
# 데이터 전처리를 위한 변환(Transform)을 정의합니다.
# 여기서는 이미지의 크기를 조정하고, 텐서로 변환하며, 정규화를 수행합니다.
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지의 크기를 224x224로 조정합니다.
    transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환합니다.
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 이미지를 정규화합니다.
])

# ImageNet 데이터셋을 로드합니다.
# 여기서는 학습 데이터셋과 검증 데이터셋을 각각 로드합니다.
train_dataset = datasets.ImageFolder(root='./data/ImageNet/train', transform=transform)
test_dataset = datasets.ImageFolder(root='./data/ImageNet/val', transform=transform)

# 데이터 로더를 생성합니다.
# 데이터 로더는 학습 중에 배치 단위로 데이터를 제공합니다.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Vision Transformer 모델을 정의합니다.
# 여기서는 timm 라이브러리에서 제공하는 vit_base_patch16_224 모델을 사용하며,
# 이 모델은 이미 ImageNet 데이터셋으로 사전 학습된 상태입니다.
model = vit_base_patch16_224(pretrained=True)
model.head = nn.Linear(model.head.in_features, 1000) # ImageNet은 1000개의 클래스를 가집니다.

# GPU를 사용할 수 있는지 확인하고, 사용 가능하면 모델을 GPU로 이동시킵니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수와 옵티마이저를 정의합니다.
# 여기서는 크로스 엔트로피 손실 함수와 Adam 옵티마이저를 사용합니다.
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# 학습 함수를 정의합니다.
# 이 함수는 주어진 에폭 수만큼 모델을 학습합니다.
def train(model, criterion, optimizer, train_loader, test_loader, epochs):
    for epoch in range(epochs):
        model.train() # 모델을 학습 모드로 설정합니다.
        for i, (images, labels) in enumerate(train_loader): # 학습 데이터셋에서 배치 단위로 데이터를 가져옵니다.
            images = images.to(device) # 이미지 데이터를 GPU로 이동시킵니다.
            labels = labels.to(device) # 레이블 데이터를 GPU로 이동시킵니다.

            outputs = model(images) # 모델을 통해 이미지 데이터를 전달하고 출력을 얻습니다.
            loss = criterion(outputs, labels) # 손실 함수를 사용하여 손실을 계산합니다.

            optimizer.zero_grad() # 옵티마이저의 그래디언트를 초기화합니다.
            loss.backward() # 손실에 대해 역전파를 수행하여 그래디언트를 계산합니다.
            optimizer.step() # 옵티마이저를 사용하여 모델의 파라미터를 업데이트합니다.

        model.eval() # 모델을 평가 모드로 설정합니다.
        correct = 0
        total = 0
        with torch.no_grad(): # 그래디언트 계산을 비활성화합니다.
            for images, labels in test_loader: # 검증 데이터셋에서 배치 단위로 데이터를 가져옵니다.
                images = images.to(device) # 이미지 데이터를 GPU로 이동시킵니다.
                labels = labels.to(device) # 레이블 데이터를 GPU로 이동시킵니다.

                outputs = model(images) # 모델을 통해 이미지 데이터를 전달하고 출력을 얻습니다.
                _, predicted = torch.max(outputs.data, 1) # 가장 높은 확률을 가진 클래스를 예측값으로 선택합니다.
                total += labels.size(0) # 전체 샘플 수를 계산합니다.
                correct += (predicted == labels).sum().item() # 정확히 예측한 샘플 수를 계산합니다.

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%') # 학습 진행 상황을 출력합니다.

# 모델 학습을 시작합니다.
train(model, criterion, optimizer, train_loader, test_loader, epochs=10)


# AI 면접 서비스 구현

요구 기능 기본
- 이미지 파일 업로드.
- 분류 값 출력.

요구 기능 심화
- 카메라 온
- TTS로 질문
- STT로 답변
- 답변 시 카메라 캡쳐
- 데이터프레임 구성 [질문, 답변, 캡쳐이미지, 평가값]